## Demo of the effectiveness of the training pipeline


In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5

In [2]:
import warnings, copy, os
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import mne
import joblib

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.tensorboard import SummaryWriter

import skorch
from skorch.classifier import NeuralNetClassifier
from skorch.helper import predefined_split
from skorch.callbacks import (LRScheduler, EpochScoring, Checkpoint, Callback,
                              TrainEndCheckpoint, LoadInitState, EarlyStopping)

from sklearn.metrics import confusion_matrix

from brainda.datasets import BNCI2014001

from brainda.paradigms import MotorImagery
from brainda.algorithms.utils.model_selection import (
    set_random_seeds, generate_kfold_indices, match_kfold_indices)
from brainda.algorithms.deep_learning import EEGNet, ShallowNet

from braindecode.models import ShallowFBCSPNet, TIDNet, EEGNetv4
from braindecode import EEGClassifier

from utils import *

In [3]:
device_id = 0
device = torch.device("cuda:{:d}".format(device_id) if torch.cuda.is_available() else "cpu")
if device != 'cpu':
    torch.backends.cudnn.benchmark = True
print("Available GPU devices: {}".format(torch.cuda.device_count()))
print("Current pytorch device: {}".format(device))

Available GPU devices: 1
Current pytorch device: cuda:0


Raw data without any preprocessing steps

- epoch: [-0.5, 4]
- srate: 250Hz
- channels: 22
- classes: left hand, right hand, feet, and tongue
- current subject: 3

Note: In our settings, `session_0` is `session_E` and `session_1` is `session_T`.

In [4]:
def raw_hook(raw, caches, verbose=False):
    # do nothing
    return raw, caches

srate = 250
dataset = BNCI2014001()
selected_channels = [
    'FZ', 'FC3', 'FC1', 'FCZ', 'FC2', 'FC4', 'C5', 'C3', 'C1', 'CZ', 'C2', 'C4', 'C6', 'CP3', 'CP1', 'CPZ', 'CP2', 'CP4', 'P1', 'PZ', 'P2', 'POZ']
start_t = dataset.events['left_hand'][1][0] - 0.5
duration = 4.5 # seconds
if start_t+ duration > dataset.events['left_hand'][1][1]:
    print("Warning: the current dataset avaliable trial duration is not long enough.")
events = ['left_hand', 'right_hand', 'feet', 'tongue']
paradigm = MotorImagery(
    channels=selected_channels, 
    srate=srate, 
    intervals=[(start_t, start_t + duration)], 
    events=events)

event_id = [dataset.events[e][0] for e in events]
paradigm.register_raw_hook(raw_hook)
X, y, meta = paradigm.get_data(dataset, subjects=[3], return_concat=True, verbose=False)
y = label_encoder(y, event_id)

### Braindecode training pipeline

Adapted from https://github.com/braindecode/braindecode/blob/master/examples/plot_bcic_iv_2a_moabb_trial.py

No bandpass filtering and exponential moving standardization

In [5]:
set_random_seeds(20200220)
n_trials, n_channels, n_samples = X.shape
labels =  np.unique(y)
n_classes = len(labels)

model = ShallowFBCSPNet(
    n_channels, n_classes,
    input_window_samples=n_samples,
    final_conv_length='auto'
)

initial_state = copy.deepcopy(model.state_dict())
criterion = nn.NLLLoss
optimizer = optim.AdamW

batch_size = 64
lr = 0.0625 * 0.01
max_epochs = 100

ckp_dirname = 'runs_training_pipeline'
verbose = True

# train set, session_T
ind = meta[meta['session']=='session_1'].sort_values(by=['session', 'run', 'trial_id']).index
trainX, trainY = np.copy(X[ind]), np.copy(y[ind])
# test set, session_E
ind = meta[meta['session']=='session_0'].sort_values(by=['session', 'run', 'trial_id']).index
validX, validY = np.copy(X[ind]), np.copy(y[ind])

trainX, validX = generate_tensors(trainX, validX, dtype=torch.float)
trainY, validY = generate_tensors(trainY, validY, dtype=torch.long)

torch.cuda.empty_cache()

model.load_state_dict(copy.deepcopy(initial_state))
train_split = predefined_split(
    skorch.dataset.Dataset(
        validX, validY))
net = EEGClassifier(model,
        criterion=criterion,
        optimizer=optimizer,
        optimizer__lr=lr,
        optimizer__weight_decay=0,
        batch_size=batch_size,
        device=device,
        train_split=train_split,
        iterator_train__shuffle=True,
        callbacks=[
            "accuracy",
            ('lr_scheduler', LRScheduler('CosineAnnealingLR', T_max=max_epochs - 1)),
        ],
        verbose=verbose)
net = net.fit(
    trainX, y=trainY, epochs=max_epochs)

  epoch    train_accuracy    train_loss    valid_accuracy    valid_loss      lr     dur
-------  ----------------  ------------  ----------------  ------------  ------  ------
      1            0.3854        1.5341            0.3194        1.7365  0.0006  0.6304
      2            0.4306        1.2407            0.3611        1.6086  0.0006  0.1703
      3            0.5660        1.0748            0.3958        1.4128  0.0006  0.1678
      4            0.7118        0.9865            0.4444        1.2122  0.0006  0.1665
      5            0.7743        0.8446            0.4861        1.1604  0.0006  0.1682
      6            0.8160        0.7972            0.4965        1.1211  0.0006  0.1670
      7            0.8542        0.6772            0.5347        1.0755  0.0006  0.1666
      8            0.8507        0.6072            0.5417        1.0613  0.0006  0.1658
      9            0.8681        0.5328            0.5486        1.0503  0.0006  0.1682
     10            0.8993       

### Our training pipeline with ShallowFBCSPNet

The difference with the above code is that we use `NeuralNetClassifierNoLog` by removing the log operation in `NeuralNetClassifier`, since ShallowFBCSPNet has already implemented `LogSoftmax` itself.

our training pipeline can achieve similar training results.

In [6]:
class NeuralNetClassifierNoLog(NeuralNetClassifier):
    def get_loss(self, y_pred, y_true, *args, **kwargs):
        return super(NeuralNetClassifier, self).get_loss(y_pred, y_true, *args, **kwargs)

set_random_seeds(20200220)
n_trials, n_channels, n_samples = X.shape
labels =  np.unique(y)
n_classes = len(labels)

model = ShallowFBCSPNet(
    n_channels, n_classes,
    input_window_samples=n_samples,
    final_conv_length='auto'
)

initial_state = copy.deepcopy(model.state_dict())
criterion = nn.NLLLoss
optimizer = optim.AdamW

batch_size = 64
lr = 0.0625 * 0.01
max_epochs = 100

ckp_dirname = 'runs_training_pipeline'
verbose = True

# train set, session_T
ind = meta[meta['session']=='session_1'].sort_values(by=['session', 'run', 'trial_id']).index
trainX, trainY = np.copy(X[ind]), np.copy(y[ind])
# test set, session_E
ind = meta[meta['session']=='session_0'].sort_values(by=['session', 'run', 'trial_id']).index
validX, validY = np.copy(X[ind]), np.copy(y[ind])

trainX, validX = generate_tensors(trainX, validX, dtype=torch.float)
trainY, validY = generate_tensors(trainY, validY, dtype=torch.long)

torch.cuda.empty_cache()
ckp = Checkpoint(dirname=ckp_dirname)
train_end_ckp = TrainEndCheckpoint(dirname=ckp_dirname)

model.load_state_dict(copy.deepcopy(initial_state))
train_split = predefined_split(
    skorch.dataset.Dataset(
        validX, validY))
net = NeuralNetClassifierNoLog(model,
        criterion=criterion,
        optimizer=optimizer,
        optimizer__lr=lr,
        optimizer__weight_decay=0,
        batch_size=batch_size,
        device=device,
        train_split=train_split,
        iterator_train__shuffle=True,
        callbacks=[
            ('train_acc', EpochScoring('accuracy', 
                                    name='train_acc', 
                                    on_train=True, 
                                    lower_is_better=False)),
            ('lr_scheduler', LRScheduler('CosineAnnealingLR', T_max=max_epochs - 1)),
            ckp,
            train_end_ckp
        ],
        verbose=verbose)
net = net.fit(
    trainX, y=trainY, epochs=max_epochs)
net.load_params(checkpoint=ckp)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.2917        1.5341       0.3194        1.7365     +  0.0006  0.2156
      2       0.4340        1.2407       0.3611        1.6086     +  0.0006  0.1708
      3       0.4931        1.0748       0.3958        1.4128     +  0.0006  0.1671
      4       0.5729        0.9865       0.4444        1.2122     +  0.0006  0.1675
      5       0.6424        0.8446       0.4861        1.1604     +  0.0006  0.1665
      6       0.7083        0.7972       0.4965        1.1211     +  0.0006  0.1682
      7       0.7431        0.6772       0.5347        1.0755     +  0.0006  0.1670
      8       0.7674        0.6072       0.5417        1.0612     +  0.0006  0.1675
      9       0.8229        0.5328       0.5486        1.0503     +  0.0006  0.1685
     10       0.8021        0.5160       0.5660        1.0188     +  0.0006 

### Our training pipeline with ShallowNet

Here we use the same training pipeline with self-implemented ShallowNet modified from ShallowFBCSPNet, replacing `conv_classifier` with `fc_layer` and removing `LogSoftmax`.

Similar results with small improvement.

In [7]:
set_random_seeds(20200220)
n_trials, n_channels, n_samples = X.shape
labels =  np.unique(y)
n_classes = len(labels)

model = ShallowNet(
    n_channels, n_samples, n_classes)

initial_state = copy.deepcopy(model.state_dict())
criterion = nn.CrossEntropyLoss
optimizer = optim.AdamW

batch_size = 64
lr = 0.0625 * 0.01
max_epochs = 100

ckp_dirname = 'runs_training_pipeline'
verbose = True

# train set, session_T
ind = meta[meta['session']=='session_1'].sort_values(by=['session', 'run', 'trial_id']).index
trainX, trainY = np.copy(X[ind]), np.copy(y[ind])
# test set, session_E
ind = meta[meta['session']=='session_0'].sort_values(by=['session', 'run', 'trial_id']).index
validX, validY = np.copy(X[ind]), np.copy(y[ind])

trainX, validX = generate_tensors(trainX, validX, dtype=torch.float)
trainY, validY = generate_tensors(trainY, validY, dtype=torch.long)

torch.cuda.empty_cache()
ckp = Checkpoint(dirname=ckp_dirname)
train_end_ckp = TrainEndCheckpoint(dirname=ckp_dirname)

model.load_state_dict(copy.deepcopy(initial_state))
train_split = predefined_split(
    skorch.dataset.Dataset(
        {'X': validX}, validY))
net = NeuralNetClassifierNoLog(model,
        criterion=criterion,
        optimizer=optimizer,
        optimizer__lr=lr,
        optimizer__weight_decay=0,
        batch_size=batch_size,
        device=device,
        train_split=train_split,
        iterator_train__shuffle=True,
        callbacks=[
            ('train_acc', EpochScoring('accuracy', 
                                    name='train_acc', 
                                    on_train=True, 
                                    lower_is_better=False)),
            ('lr_scheduler', LRScheduler('CosineAnnealingLR', T_max=max_epochs - 1)),
            ckp,
            train_end_ckp
        ],
        verbose=verbose)
net = net.fit(
    {'X': trainX}, y=trainY, epochs=max_epochs)
net.load_params(checkpoint=ckp)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.2222        1.6694       0.2951        1.7453     +  0.0006  0.6310
      2       0.4306        1.2924       0.3403        1.5502     +  0.0006  0.2201
      3       0.5139        1.1181       0.3819        1.4406     +  0.0006  0.2139
      4       0.5486        1.0909       0.4549        1.3102     +  0.0006  0.2147
      5       0.6042        0.9650       0.4653        1.1612     +  0.0006  0.2130
      6       0.6632        0.8896       0.5243        1.1059     +  0.0006  0.2147
      7       0.7326        0.7701       0.5312        1.0827     +  0.0006  0.2155
      8       0.7569        0.7109       0.5451        1.0511     +  0.0006  0.2136
      9       0.7847        0.6344       0.5556        1.0688        0.0006  0.2134
     10       0.8229        0.5440       0.5625        1.0805        0.0006 

### Our training pipeline with EEGNetv4

In [8]:
class NeuralNetClassifierNoLog(NeuralNetClassifier):
    def get_loss(self, y_pred, y_true, *args, **kwargs):
        return super(NeuralNetClassifier, self).get_loss(y_pred, y_true, *args, **kwargs)

set_random_seeds(20200220)
n_trials, n_channels, n_samples = X.shape
labels =  np.unique(y)
n_classes = len(labels)

model = EEGNetv4(n_channels, n_classes, 
            input_window_samples=n_samples, 
            F1=8,
            D=2,
            F2=16,
            kernel_length=64,
            drop_prob=0.5)

initial_state = copy.deepcopy(model.state_dict())
criterion = nn.NLLLoss
optimizer = optim.AdamW

batch_size = 64
lr = 0.01
max_epochs = 100

ckp_dirname = 'runs_training_pipeline'
verbose = True

# train set, session_T
ind = meta[meta['session']=='session_1'].sort_values(by=['session', 'run', 'trial_id']).index
trainX, trainY = np.copy(X[ind]), np.copy(y[ind])
# test set, session_E
ind = meta[meta['session']=='session_0'].sort_values(by=['session', 'run', 'trial_id']).index
validX, validY = np.copy(X[ind]), np.copy(y[ind])

trainX, validX = generate_tensors(trainX, validX, dtype=torch.float)
trainY, validY = generate_tensors(trainY, validY, dtype=torch.long)

torch.cuda.empty_cache()
ckp = Checkpoint(dirname=ckp_dirname)
train_end_ckp = TrainEndCheckpoint(dirname=ckp_dirname)

model.load_state_dict(copy.deepcopy(initial_state))
train_split = predefined_split(
    skorch.dataset.Dataset(
        validX, validY))
net = NeuralNetClassifierNoLog(model,
        criterion=criterion,
        optimizer=optimizer,
        optimizer__lr=lr,
        optimizer__weight_decay=0,
        batch_size=batch_size,
        device=device,
        train_split=train_split,
        iterator_train__shuffle=True,
        callbacks=[
            ('train_acc', EpochScoring('accuracy', 
                                    name='train_acc', 
                                    on_train=True, 
                                    lower_is_better=False)),
            ('lr_scheduler', LRScheduler('CosineAnnealingLR', T_max=max_epochs - 1)),
            ckp,
            train_end_ckp
        ],
        verbose=verbose)
net = net.fit(
    trainX, y=trainY, epochs=max_epochs)
net.load_params(checkpoint=ckp)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.2917        1.5321       0.3681        1.3416     +  0.0100  0.3494
      2       0.4132        1.2833       0.3889        1.3704        0.0100  0.2431
      3       0.4757        1.1720       0.4861        1.3713        0.0100  0.2401
      4       0.5486        1.0289       0.4583        1.5700        0.0100  0.2401
      5       0.6319        0.8532       0.4132        1.8973        0.0100  0.2407
      6       0.6701        0.8333       0.4167        2.0515        0.0099  0.2400
      7       0.7188        0.7220       0.4583        1.7437        0.0099  0.2409
      8       0.7639        0.6607       0.4965        1.4679        0.0099  0.2392
      9       0.7778        0.5817       0.5903        1.2075     +  0.0098  0.2407
     10       0.7500        0.6228       0.6181        0.9796     +  0.0098 

### Our training pipeline with EEGNet

In [9]:
set_random_seeds(20200220)
n_trials, n_channels, n_samples = X.shape
labels =  np.unique(y)
n_classes = len(labels)

model = EEGNet(n_channels, n_samples, n_classes,
            time_kernel=(8, (1, 64), (1, 1)),
            D=2,
            pool_kernel1=((1, 4), (1, 4)),
            separa_kernel=(16, (1, 16), (1, 1)),
            pool_kernel2=((1, 8), (1, 8)),
            depthwise_norm_rate=1,
            fc_norm_rate=0.25,
            dropout_rate=0.5)

initial_state = copy.deepcopy(model.state_dict())
criterion = nn.CrossEntropyLoss
optimizer = optim.AdamW

batch_size = 64
lr = 0.01
max_epochs = 100

ckp_dirname = 'runs_training_pipeline'
verbose = True

# train set, session_T
ind = meta[meta['session']=='session_1'].sort_values(by=['session', 'run', 'trial_id']).index
trainX, trainY = np.copy(X[ind]), np.copy(y[ind])
# test set, session_E
ind = meta[meta['session']=='session_0'].sort_values(by=['session', 'run', 'trial_id']).index
validX, validY = np.copy(X[ind]), np.copy(y[ind])

trainX, validX = generate_tensors(trainX, validX, dtype=torch.float)
trainY, validY = generate_tensors(trainY, validY, dtype=torch.long)

torch.cuda.empty_cache()
ckp = Checkpoint(dirname=ckp_dirname)
train_end_ckp = TrainEndCheckpoint(dirname=ckp_dirname)

model.load_state_dict(copy.deepcopy(initial_state))
train_split = predefined_split(
    skorch.dataset.Dataset(
        validX, validY))
net = NeuralNetClassifier(model,
        criterion=criterion,
        optimizer=optimizer,
        optimizer__lr=lr,
        optimizer__weight_decay=0,
        batch_size=batch_size,
        device=device,
        train_split=train_split,
        iterator_train__shuffle=True,
        callbacks=[
            ('train_acc', EpochScoring('accuracy', 
                                    name='train_acc', 
                                    on_train=True, 
                                    lower_is_better=False)),
            ('lr_scheduler', LRScheduler('CosineAnnealingLR', T_max=max_epochs - 1)),
            ckp,
            train_end_ckp
        ],
        verbose=verbose)
net = net.fit(
    trainX, y=trainY, epochs=max_epochs)
net.load_params(checkpoint=ckp)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.2396        1.5590       0.3333        1.3508     +  0.0100  0.1676
      2       0.3299        1.4080       0.3576        1.3181     +  0.0100  0.1197
      3       0.4167        1.2456       0.4097        1.2868     +  0.0100  0.1186
      4       0.4479        1.1780       0.4271        1.2497     +  0.0100  0.1172
      5       0.5486        1.1128       0.4688        1.2172     +  0.0100  0.1175
      6       0.5694        1.0116       0.4826        1.2103     +  0.0099  0.1182
      7       0.6076        0.9260       0.5035        1.1957     +  0.0099  0.1176
      8       0.6632        0.8628       0.5382        1.1804     +  0.0099  0.1191
      9       0.6667        0.8079       0.4965        1.2901        0.0098  0.1179
     10       0.6701        0.7476       0.5451        1.2194        0.0098 

In [10]:
class NeuralNetClassifierNoLog(NeuralNetClassifier):
    def get_loss(self, y_pred, y_true, *args, **kwargs):
        return super(NeuralNetClassifier, self).get_loss(y_pred, y_true, *args, **kwargs)

set_random_seeds(20200220)
n_trials, n_channels, n_samples = X.shape
labels =  np.unique(y)
n_classes = len(labels)

model = TIDNet(n_channels, n_classes, 
            input_window_samples=n_samples,
            s_growth=24,
            t_filters=32, 
            temp_layers=2, 
            spat_layers=2, 
            pooling=15, 
            temp_span=0.05, 
            bottleneck=3)

initial_state = copy.deepcopy(model.state_dict())
criterion = nn.NLLLoss
optimizer = optim.AdamW

batch_size = 64
lr = 0.001
max_epochs = 100

ckp_dirname = 'runs_training_pipeline'
verbose = True

# train set, session_T
ind = meta[meta['session']=='session_1'].sort_values(by=['session', 'run', 'trial_id']).index
trainX, trainY = np.copy(X[ind]), np.copy(y[ind])
# test set, session_E
ind = meta[meta['session']=='session_0'].sort_values(by=['session', 'run', 'trial_id']).index
validX, validY = np.copy(X[ind]), np.copy(y[ind])

trainX, validX = generate_tensors(trainX, validX, dtype=torch.float)
trainY, validY = generate_tensors(trainY, validY, dtype=torch.long)

torch.cuda.empty_cache()
ckp = Checkpoint(dirname=ckp_dirname)
train_end_ckp = TrainEndCheckpoint(dirname=ckp_dirname)

model.load_state_dict(copy.deepcopy(initial_state))
train_split = predefined_split(
    skorch.dataset.Dataset(
        validX, validY))
net = NeuralNetClassifierNoLog(model,
        criterion=criterion,
        optimizer=optimizer,
        optimizer__lr=lr,
        optimizer__weight_decay=0,
        batch_size=batch_size,
        device=device,
        train_split=train_split,
        iterator_train__shuffle=True,
        callbacks=[
            ('train_acc', EpochScoring('accuracy', 
                                    name='train_acc', 
                                    on_train=True, 
                                    lower_is_better=False)),
            ('lr_scheduler', LRScheduler('CosineAnnealingLR', T_max=max_epochs - 1)),
            ckp,
            train_end_ckp
        ],
        verbose=verbose)
net = net.fit(
    trainX, y=trainY, epochs=max_epochs)
net.load_params(checkpoint=ckp)

  epoch    train_acc    train_loss    valid_acc    valid_loss    cp      lr     dur
-------  -----------  ------------  -----------  ------------  ----  ------  ------
      1       0.2986        1.9316       0.3090        2.0845     +  0.0010  3.1516
      2       0.4271        1.5461       0.2812        2.5431        0.0010  2.1912
      3       0.4549        1.2975       0.3715        1.9438     +  0.0010  2.2093
      4       0.5938        0.9878       0.3889        2.0292        0.0010  2.1937
      5       0.6493        0.8703       0.4097        1.7038     +  0.0010  2.2328
      6       0.7083        0.7619       0.4236        1.7468        0.0010  2.2357
      7       0.7535        0.5907       0.3993        1.8018        0.0010  2.2300
      8       0.7917        0.5575       0.4236        1.7799        0.0010  2.2154
      9       0.8576        0.4292       0.4201        1.8375        0.0010  2.2094
     10       0.8750        0.3498       0.4132        1.9134        0.0010 